# Plugin Development

## Introduction

A CIS ‘plugin’ is just a piece of Python code which CIS can read and understand to extend its functionality

Today the focus is on data reading plugins which allow CIS to read data sources that it wouldn’t otherwise understand

More specifically a plugin is a Python class which subclasses a template class – so that CIS can assume certain things about the plugin.

In [ ]:
from cis.data_io.products.AProduct import AProduct

class my_plugin(AProduct)

One of the first things to consider is which type of file our plugin is going to be aimed at reading. It is advisable to not make the definition too broad, it’s easy to have multiple plugins so don’t try and over complicate the plugin by having it read many different types of file. Roughly, one plugin should describe a set of data with the same metadata.

Each data reading plugin must include the following methods:
 - `get_file_signature(self)`
 - `create_coords(self, filenames)`
 - `create_data_object(self, filenames, variable)`

## Setup

First of all we need an ‘environment’ in which to create our plugins, we have a few options:
 - Basic text editor,
 - Python aware text editor,
 - Or an Integrated Development Environment (IDE)


We also need to point CIS to our new plugins, just set `CIS_PLUGIN_HOME` to the right folder

## Tutorial

For this tutorial we’re going to work through the CIS data reading plugin. This already exists within CIS, but it serves as a nice example for making your own.

It’s useful to have a look inside one of the CIS output files to see the structure
 - Note that it shouldn’t be necessary to actually worry about the I/O most of the time though


Let’s start with the `get_file_signature` method.

This method is used by CIS to determine which plugins to use automatically.

It should return a list of regular expressions which match (as closely as possible) the files which the plugin is designed to read.

Since the CIS plugin is designed to read any data which CIS produces, the signature matches any file which starts with cis- and ends with .nc:

In [ ]:
def get_file_signature(self):
    return [r'cis\\-.\*\\.nc']

The next step is to complete the `create_coords()` method.

CIS uses this method to create a set of coordinates from the data, so it needs to return any appropriate coordinates in the shape that CIS expects it (an `UngriddedCoordinates` object).

There are a number of low-level data reading routines within CIS that can help you read in your data. For the CIS plugin (which is reading netCDF data) we use two methods from the `cis.data_io.netcdf` module:
 1. `read_many_files_individually`
 2. `get_metadata`. 

In [ ]:
def create_coords(self, filenames):
    from cis.data_io.netcdf import read_many_files_individually, get_metadata
    from cis.data_io.Coord import Coord, CoordList
    from cis.data_io.ungridded_data import UngriddedCoordinates

We also import the `Coord` data type, which is where we store the coordinates that we’ve read, and `UngriddedCoordinates` - which is what we return to CIS.

Next, we create a list of netCDF variable names which we know are stored in our file and send that to the file reading routine:

In [ ]:
    var_data = read_many_files_individually(filenames, ["longitude","latitude", "time"])

Then we create a `CoordList` to store our coordinates in, a `Coord` for each of those coordinate variables, and then just give them a short label for plotting purposes (x,y,z etc).

it is strongly advisable that you use the standard definitions used below for your axis definitions (and use z for altitude and p for pressure).

In [ ]:
    lon = Coord(var_data["longitude"], get_metadata(var_data["longitude"][0]), axis="x")
    lat = Coord(var_data["latitude"], get_metadata(var_data["latitude"][0]), axis="y")
    time = Coord(var_data["time"], get_metadata(var_data["time"][0]), axis="t")
    coords = CoordList([lat, lon, time])

That’s it, now we can return those coordinates in a way that CIS will understand:

In [ ]:
    return UngriddedCoordinates(coords)

So we should have:

In [ ]:
def create_coords(self, filenames):
    from cis.data_io.netcdf import read_many_files_individually, get_metadata
    from cis.data_io.Coord import Coord, CoordList
    from cis.data_io.ungridded_data import UngriddedCoordinates

    var_data = read_many_files_individually(filenames, ["longitude", "latitude", "time"])

    lon = Coord(var_data["longitude"], get_metadata(var_data["longitude"][0]), axis="x")
    lat = Coord(var_data["latitude"], get_metadata(var_data["latitude"][0]), axis="y")
    time = Coord(var_data["time"], get_metadata(var_data["time"][0]), axis="t")
    coords = CoordList([lat, lon, time])

    return UngriddedCoordinates(coords)


The last method we have to write is the `create_data_object()` method. 

This is used by CIS to pull together the coordinates and a particular data variable into an `UngriddedData` object. 

It’s even simpler than the previous method. We can use the same `read_many_files_individually` method as we did before, and this time pass it the variable the user has asked for as well:

In [ ]:
def create_data_object(self, filenames, variable):
    from cis.data_io.netcdf import read_many_files_individually, get_metadata
    from cis.data_io.Coord import Coord, CoordList
    from cis.data_io.ungridded_data import UngriddedData

    var_data = read_many_files_individually(filenames, ["longitude", "latitude", "time", variable])


Then we create the coordinates using the same code we’ve just written:

In [ ]:
    lon = Coord(var_data["longitude"], get_metadata(var_data["longitude"][0]), axis="x")
    lat = Coord(var_data["latitude"], get_metadata(var_data["latitude"][0]), axis="y")
    time = Coord(var_data["time"], get_metadata(var_data["time"][0]), axis="t")
    coords = CoordList([lat, lon, time])


And finally we return the ungridded data, this combines the coordinates from the file and the variable requested by the user:

In [ ]:
    usr_var_data = var_data[variable]

    return UngriddedData(usr_var_data, get_metadata(usr_var_data[0]), coords)


## Using your plugin

Using your plugin is easy. If your signature uniquely matches an input file CIS will use it automatically.

Otherwise you will need to manually specify it:

In [ ]:
$ cis plot AOD550:cis-aerosol_cci_subset_Alaska.nc:product=my_product

## Sharing your plugin

This is really easy! 

Just go to http://www.cistools.net/plugins and submit your plugin.

If it becomes popular we will look to roll it into the main CIS products (crediting the author of course!)